<a href="https://colab.research.google.com/github/keerthana-nc/Deeplearning-LSTM/blob/main/VANILLA_LSTM_3layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas scipy


importing the necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')
import warnings
warnings.filterwarnings('ignore')
from time import time
import matplotlib.ticker as tkr
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from statsmodels.tsa.stattools import pacf
%matplotlib inline

import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

In [ ]:
from scipy.stats import skew

uploads the dataset

creates a dataframe called 'df'

df.head function prints the first five rows of the dataset the dataframe 'df' is referring to.

In [ ]:
from google.colab import files
uploaded = files.upload()

filename = next(iter(uploaded))
df = pd.read_csv(filename)
print('Number of rows and columns:', df.shape)
df.head(5)

Saving RCD_200_10_withdate&time.csv to RCD_200_10_withdate&time.csv
Number of rows and columns: (35040, 2)


,date_time,Actual Power
0,01-01-2020 00:00,3.2665
1,01-01-2020 00:15,3.9529
2,01-01-2020 00:30,3.6021
3,01-01-2020 00:45,3.5990
4,01-01-2020 01:00,2.6076


two columns called date and time are combined as one column

converting date and time values into numeric values

In [ ]:
'''
import pandas as pd

# Assuming 'column_name' is the name of the column you want to convert
df['date_time'] = df['date_time'].astype('float32')
'''

"\nimport pandas as pd\n\n# Assuming 'column_name' is the name of the column you want to convert\ndf['date_time'] = df['date_time'].astype('float32')\n"

In [ ]:
# Convert the "date_time" column to a pandas DateTime data type
df['date_time'] = pd.to_datetime(df['date_time'])

# Extract the Unix timestamp from the DateTime values
timestamps = df['date_time'].astype(np.int64) / 10**9

# Convert the timestamps to a NumPy array of floats
timestamps_np = np.array(timestamps, dtype=np.float32)

In [ ]:

# Select columns to normalize
cols_to_normalize = ['Actual Power']

# Normalize the selected columns using MinMaxScaler
scaler = MinMaxScaler()
df[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])

# Print the first few rows of the normalized dataset
print(df.head())


            date_time  Actual Power
0 2020-01-01 00:00:00        0.4521
1 2020-01-01 00:15:00        0.5967
2 2020-01-01 00:30:00        0.5228
3 2020-01-01 00:45:00        0.5221
4 2020-01-01 01:00:00        0.3133


applying logarithmic tranformation to

In [ ]:
epsilon = 1e-8  # Small constant to avoid division by zero

# Add epsilon and calculate skewness
df['Active Power'] = np.log(df['Active Power'] + epsilon)
#df['Sub_metering_2'] = np.log(df['Sub_metering_2'] + epsilon)
#df['Sub_metering_3'] = np.log(df['Sub_metering_3'] + epsilon)

skewness = df[['Active Power']].skew()
print('Skewness:')
print(skewness)

Skewness:
Active Power   -6.5089
dtype: float64


In [ ]:
print('Number of rows and columns after removing missing values:', df.shape)
print('The time series starts from: ', df.date_time.min())
print('The time series ends on: ', df.date_time.max())

Number of rows and columns after removing missing values: (35040, 2)
The time series starts from:  2020-01-01 00:00:00
The time series ends on:  2020-12-30 23:42:00


In [ ]:
print(df.isna().sum())

date_time       0
Actual Power    0
dtype: int64


working on dataset

In [ ]:
# Split the data into input (X) and target (Y)
X = df[['date_time', 'Actual Power']].values


In [ ]:
X = np.column_stack((timestamps_np, df['Actual Power'].values))


In [ ]:
# Extract the "total power" column as the target variable
total_power = df['Actual Power'].values


In [ ]:
# Split the dataset into input features (X) and target variables (y)
'''
X = df[cols_to_normalize].values
'''

# Shift the target variables by one time step to create forecasting scenario
y = np.roll(total_power, -1)

In [ ]:
# Remove the last row from X and y since y has a shifted row
X = X[:-1]
y = y[:-1]

In [ ]:
# Ensure the shapes of X and y are compatible
print(X.shape)  # (n_samples, 2)
print(y.shape)  # (n_samples,)

(35039, 2)
(35039,)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
# Reshape the input data to match the LSTM layer's input shape
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=64, return_sequences=True))  # Add the third LSTM layer
model.add(LSTM(units=32))  # Add the fourth LSTM layer
model.add(Dense(units=1))  # Three output units for the three attributes

In [ ]:
print(X_train.dtype)

float64


In [ ]:
print('y_test shape:', y_test.shape)
y_pred = model.predict(X_test)
print('y_pred shape:', y_pred.shape)

y_test shape: (7008,)
219/219 [==============================] - 2s 4ms/step
y_pred shape: (7008, 1)


In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=64)

# Predict using the trained model
y_pred = model.predict(X_test)



Epoch 1/25
438/438 [==============================] - 12s 13ms/step - loss: 0.0148
Epoch 2/25
438/438 [==============================] - 7s 17ms/step - loss: 0.0037
Epoch 3/25
438/438 [==============================] - 6s 14ms/step - loss: 0.0042
Epoch 4/25
438/438 [==============================] - 7s 15ms/step - loss: 0.0036
Epoch 5/25
438/438 [==============================] - 6s 13ms/step - loss: 0.0034
Epoch 6/25
438/438 [==============================] - 7s 16ms/step - loss: 0.0034
Epoch 7/25
438/438 [==============================] - 8s 17ms/step - loss: 0.0034
Epoch 8/25
438/438 [==============================] - 8s 17ms/step - loss: 0.0034
Epoch 9/25
438/438 [==============================] - 7s 16ms/step - loss: 0.0036
Epoch 10/25
438/438 [==============================] - 5s 13ms/step - loss: 0.0032
Epoch 11/25
438/438 [==============================] - 6s 15ms/step - loss: 0.0028
Epoch 12/25
438/438 [==============================] - 6s 14ms/step - loss: 0.0026
Epoch 13/25


In [ ]:
# Calculate the RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:', rmse)
mae = mean_absolute_error(y_test, y_pred)
print('MAE:',mae)

RMSE: 0.08757939146106081
MAE: 0.06802532371844708
